# 네이버 뉴스 데이터 가져오기

> 1. 네이버 오픈 API로 기사 URL 크롤링
> 2. bs4 + selenium으로 기사 제목 및 내용 크롤링

In [6]:
import os
import sys
import urllib.request
import requests

news_data = []

page_count = 3

client_id = "FZ67wgW6UAbI4UUIELOl"
client_secret = "PPyO_d8wIQ" # 발급받은 client_secret
encText = urllib.parse.quote("파이썬")

for idx in range(page_count):
    url = "https://openapi.naver.com/v1/search/news?query=" + encText +"&start=" + str(idx * 10 +1)# json 결과
    # url = "https://openapi.naver.com/v1/search/blog.xml?query=" + encText # xml 결과


    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id",client_id)
    request.add_header("X-Naver-Client-Secret",client_secret)
    response = urllib.request.urlopen(request)
    rescode = response.getcode()


    if(rescode==200):
        # response_body = response.read()
        result = requests.get(response.geturl(),
                            headers={"X-Naver-Client-Id":client_id,
                                    "X-Naver-Client-Secret":client_secret}
                            )
        news_data.append(result.json())
        # print(response_body.decode('utf-8'))
    else:
        print("Error Code:" + rescode)

### 네이버 OPEN API를 통해 가져온 데이터 확인하기.

In [7]:
news_data[0]['items'][0]['link']

'http://www.newsway.co.kr/news/view?tp=1&ud=2022011317585132477'

### 가져온 URL이 네이버 뉴스인지 확인하기.

In [12]:
naver_news_link = []

for page in news_data:

    page_news_link = []

    for item in page['items']:
        link = item['link']
        if "naver" in link:
            page_news_link.append(link)

    naver_news_link.append(page_news_link)

for page in naver_news_link:
    for link in page:   
      print(link)


https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=105&oid=009&aid=0004908366
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=105&oid=092&aid=0002244854
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=101&oid=011&aid=0004007576
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=102&oid=030&aid=0002993463
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=105&oid=138&aid=0002116992
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=101&oid=011&aid=0004006907
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=101&oid=029&aid=0002715336
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=105&oid=092&aid=0002244490


### 네이버 뉴스 기사 크롤링!

In [5]:
import pandas as pd
import numpy as np
from selenium import webdriver
from tqdm import tqdm_notebook
import requests
import pickle
import re
import ast

from bs4 import BeautifulSoup 
from urllib.request import urlopen
import urllib
import time

In [10]:
# 가상 크롬드라이버를 불러옴.
# 윈도우 10의 경우 chromedriver.exe
driver = webdriver.Chrome("./chromedriver.exe")

C:\Users\jx778\AppData\Local\Temp/ipykernel_7068/887318981.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("./chromedriver.exe")


In [14]:
naver_news_title = []
naver_news_content = []


for n in tqdm_notebook(range(len(naver_news_link))):
    #print(n)
    news_page_title = []
    news_page_content = []
    
    for idx in tqdm_notebook(range(len(naver_news_link[n]))):
        
        
    ########### 긁어온 URL로 접속하기 ############    
        try:
            driver.get(naver_news_link[n][idx])
            print(naver_news_link[n][idx])
            
        except:
            print("Timeout!")
            continue
        
        
        try:
            response = driver.page_source
            
        except UnexpectedAlertPresentException:
            driver.switch_to_alert().accept()
            print("게시글이 삭제된 경우입니다.")
            continue
        
        soup = BeautifulSoup(response, "html.parser")
        
        ###### 뉴스 타이틀 긁어오기 ######
        
        title = None
        
        try:
            item = soup.find('div', class_="article_info")
            title = item.find('h3', class_="tts_head").get_text()
            #print(title)

        except:
            title = "OUTLINK"
        
        #print(title)
        news_page_title.append(title)
        
        
        ###### 뉴스 본문 긁어오기 ######
        
        doc = None
        text = ""
                
        data = soup.find_all("div", {"class" : "_article_body_contents"})
        if data:
            for item in data:

                text = text + str(item.find_all(text=True)).strip()
                text = ast.literal_eval(text)
                doc = ' '.join(text)
   
        else:
            doc = "OUTLINK"
            
        news_page_content.append(doc.replace('\n', ' '))

                
    naver_news_title.append(news_page_title)
    naver_news_content.append(news_page_content)

    time.sleep(2)
    
    
print(naver_news_title[0])
print("==================================")
print(naver_news_content[0])

C:\Users\jx778\AppData\Local\Temp/ipykernel_7068/3096456725.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for n in tqdm_notebook(range(len(naver_news_link))):


  0%|          | 0/3 [00:00<?, ?it/s]

C:\Users\jx778\AppData\Local\Temp/ipykernel_7068/3096456725.py:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for idx in tqdm_notebook(range(len(naver_news_link[n]))):


  0%|          | 0/3 [00:00<?, ?it/s]

https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=105&oid=009&aid=0004908366
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=105&oid=092&aid=0002244854
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=101&oid=011&aid=0004007576


  0%|          | 0/3 [00:00<?, ?it/s]

https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=102&oid=030&aid=0002993463
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=105&oid=138&aid=0002116992
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=101&oid=011&aid=0004006907


  0%|          | 0/2 [00:00<?, ?it/s]

https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=101&oid=029&aid=0002715336
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=105&oid=092&aid=0002244490
['엘리스, ‘K-디지털 크레딧’ 과정 교육생 모집', '엘리스, ‘K-디지털 크레딧’ 과정 교육생 모집', '엘리스, ‘K-디지털 크레딧’ 과정 교육생 모집···"100% 무료 수강"']
['   본문 내용     TV플레이어     // TV플레이어     // flash 오류를 우회하기 위한 함수 추가 function _flash_removeCallback() {}      디지털 교육 실습 플랫폼 엘리스(대표 김재원)가 무료 수강이 가능한 ‘ K- 디지털 크레딧( K-Digital   Credit ,  KDC )’의 새로운 교육생을 모집한다.  K- 디지털 크레딧은 고용노동부가 청년, 중장년 구직자의 디지털 기초역량을 강화하기 위해 마련한 직업훈련의 일환으로 코딩, 빅데이터, 앱 개발 등 디지털 신기술 분야에서 입문 과정부터 교육받을 수 있다. 정부가 전체 교육비의  90 %를 지원하며, 이수 후엔  10 % 자부담비도  100 % 환급된다.  엘리스는 탁월한 플랫폼 기술력과 운영 노하우를 집약해 괄목할 만한 성과를 거뒀다. 지난해  KDC  교육생은  2020 년 대비  150 % 이상 증가했고, 교육 만족도는 5점 만점에 약  4.9 점을 기록하며 가파른 증가세를 보이고 있다. 엘리스 교육과정의 특징은 실습 중심 교육으로 불필요한 요소를 최소화해 동기부여와 학습효과를 극대화했다는 점이다. 로그인만 하면 단 3초 만에 바로 코딩 학습이 가능하며, 화면 전환 없이 한 화면에서 영상을 보며 코드 작성 및 검토도 할 수 있다. 클릭 한 번으로 작성한 코드를 모두 확인받고, 1:1 튜터링을 통해 현직 개발자과 신속하게 질의응답하며 교육에 대한 이해도

### 크롤링 결과 확인하기.

In [15]:
print(naver_news_title[0])

['엘리스, ‘K-디지털 크레딧’ 과정 교육생 모집', '엘리스, ‘K-디지털 크레딧’ 과정 교육생 모집', '엘리스, ‘K-디지털 크레딧’ 과정 교육생 모집···"100% 무료 수강"']


In [16]:
print(naver_news_content[0])

['   본문 내용     TV플레이어     // TV플레이어     // flash 오류를 우회하기 위한 함수 추가 function _flash_removeCallback() {}      디지털 교육 실습 플랫폼 엘리스(대표 김재원)가 무료 수강이 가능한 ‘ K- 디지털 크레딧( K-Digital   Credit ,  KDC )’의 새로운 교육생을 모집한다.  K- 디지털 크레딧은 고용노동부가 청년, 중장년 구직자의 디지털 기초역량을 강화하기 위해 마련한 직업훈련의 일환으로 코딩, 빅데이터, 앱 개발 등 디지털 신기술 분야에서 입문 과정부터 교육받을 수 있다. 정부가 전체 교육비의  90 %를 지원하며, 이수 후엔  10 % 자부담비도  100 % 환급된다.  엘리스는 탁월한 플랫폼 기술력과 운영 노하우를 집약해 괄목할 만한 성과를 거뒀다. 지난해  KDC  교육생은  2020 년 대비  150 % 이상 증가했고, 교육 만족도는 5점 만점에 약  4.9 점을 기록하며 가파른 증가세를 보이고 있다. 엘리스 교육과정의 특징은 실습 중심 교육으로 불필요한 요소를 최소화해 동기부여와 학습효과를 극대화했다는 점이다. 로그인만 하면 단 3초 만에 바로 코딩 학습이 가능하며, 화면 전환 없이 한 화면에서 영상을 보며 코드 작성 및 검토도 할 수 있다. 클릭 한 번으로 작성한 코드를 모두 확인받고, 1:1 튜터링을 통해 현직 개발자과 신속하게 질의응답하며 교육에 대한 이해도를 높인다.  엘리스는 참여 교육 업체 중 최대 규모인  37 개 과목을 운영하며 뛰어난 교육 역량과 전문성을 인정받았다. 파이썬,  SQL , 인공지능 분야에서 코딩 경험이 전혀 없는 입문자를 위한 과정에서 고급 과정까지 다채롭게 구성돼 원하는 과목을 자유롭게 선택하면 된다. 세부 과목은 ▲파이썬 기초 프로그래밍 ▲ SQL  기초 ▲ JAVA  웹 개발 ▲웹 페이지 제작 ▲파이썬 데이터 분석 ▲R 데이터 분석 ▲인공지능 수학 ▲심화 머신러닝/딥러닝 ▲코딩 테스트 대비 알고리즘 ▲리눅스 ▲ git ,

In [17]:
print(len(naver_news_title[0]))
print(len(naver_news_content[0]))

3
3


### 크롤링 결과 저장하기.

In [ ]:
with open("naver_news_title.pk", 'wb') as f:
    pickle.dump(naver_news_title, f)

with open("naver_news_content.pk",'wb') as f:
    pickle.dump(naver_news_content, f)